<a href="https://colab.research.google.com/github/eunzzae/Study_RS/blob/main/TF_IDF_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* kss는 한국어 문장 분리 모듈

In [2]:
!pip install kss

In [3]:
import os
import kss
import re

In [4]:
!pip install mecab-python3

In [5]:
# 형태소 기반 토크나이징(Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [6]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!python3 -m pip install --upgrade pip
!python3 -m pip install konlpy       # Python 3.x
!python3 -m pip install mecab-python3

# Mecab 설치 스크립트 실행
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,079 kB]
Fetched 1,308 kB in 2s (597 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... 

## Read Dataset

* Github의 data 디렉토리에서 news_sample.txt 파일을 다운받아 분석에 사용

In [7]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
  # 특수문자를 전처리하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)

    # removal_list에 있는 모든 문자를 공백으로 변환
    removal_list = "특정 제거할 문자들"  # 예를 들어, '[](){}'
    translation_table = str.maketrans(removal_list, ' '*len(removal_list))
    sentence = sentence.translate(translation_table)

    sentence = sentence.strip()

    return sentence

def cleansing_numbers(sentence):
  # 숫자를 전처리하는 함수
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)

    return sentence

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs('안녕하세요, Mecab 설치를 확인합니다.'))


['안녕', '하', '세요', ',', 'Mecab', '설치', '를', '확인', '합니다', '.']


In [9]:
# 각자 작업 환경에 맞는 경로를 지정해주세요.
data_path = '/content/drive/MyDrive/Colab Notebooks/패스트캠퍼스/[딥러닝] 추천시스템/data'
all_sentences = []

with open(os.path.join(data_path,'news_sample.txt'), 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f.readlines()):
      print(f"---문서 {idx} 번 ---")
      preprocessed = cleansing_numbers(line)
      preprocessed = cleansing_special(preprocessed)

      # 명사만 추출
      preprocessed_news = ' '.join(list(set(mecab.nouns(preprocessed))))
      # 문장 분리해서 사용하는 방법
      # preprocessed_news = ' '.join(kss.split_sentences(preprocessed))
      print(preprocessed_news)
      all_sentences.append(preprocessed_news)

---문서 0 번 ---
때 전력 인맥 미국 바탕 청산 것 청사진 수 일 논리 육 소망 대결 현안 좌절 성과 조기 휘하 글로벌 국민 슬기 기억 새해 갈등 선 군 듯 사람 관행 무엇 기초 평화 방향 도민 극복 다이 위기 지혜 역량 욕 번지 생명력 대비 공감대 기반 관계 전환점 립 당위 희망 뒤 선진국 협의 다경 신 시련 이것 해다 공유 포함 인사 회 그동안 이 치적 본 꿈 역사 뜻 퇴행 낭비성 한미 상화 민족 아침 여건 열차 련 대화 평창 미래 우리 사무소 햇살 중요 간 다음 건국 변수 조화 현상 속 불씨 대 열쇠 흑인 남북 금강산 출발 기대 통행 질 구체 달성 표 취임 업그레이드 변화 활용 집행 단체장 활력 얼마 능력 북 결 치 과 모색 개성관광 근 치력 태양 한국 당국 유념 성공 등 경기 화해 지방 고유 경의선 인구 개성 기업 지역 국호 번 민 추진력 원년 단계 임행 년 약점 떼 시간 외교 틀 색 경 부가 시대 기회비용 단체 암울 질주 발휘 도약 적재적소 배치 가치 동해 인재 극대 치행 운행 쇠 땅 미증유 글 시 절망 통일 바람 처음 오바마 성향 몸 도시 몫 파악 행 저력 생산 중단 다선 실행 남북한 해결 단추 강원도 광채 발굴 지난해 화두 목적 음모 날 지 바다 유치 핵 올림픽 책 경협 면적 유능 증오 후 구축 후보 도전 국익 동기 예산 최대 단절 부여 암투 축년 폐쇄 추진 방법 불신 한반도 회한 국내 인원 동계 대통령 나열 올해 발전 혼란 직무 피 마음 코드 고난 생력 공직 점
---문서 1 번 ---
지금 체장 약속 바탕 분담 시책 것 부지런 수 일 공조 일선 체 어깨 청취 일조 과시 성과 인센티브 시스템 세 역동 계축 회장 이주희 준수 주력 행복 새해 갈등 취약 군 운영 역 난관 내부 공 걸 기초 중국 도민 다사다난 서비스 체계 역량 모습 반영 김진선지사 소리 라인 요구 희망 뒤 애인 핫 협의 도장 부회장 현재 김용주 사회 그동안 월 북경 이 이사 꿈 전국 인과 지도 건강 마음가짐 종목 임직원 가운데 무분별 엘리트 최일선 무대 체전 대화 체육 우리 간

# Scikit_learn을 사용한 TF-IDF

* 한글자 단어 제외
* https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
* Useful option: tokenizer, stop_words

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [11]:
print(len(all_sentences))

20


In [12]:
!pip install -U scikit-learn

In [13]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

word2id = defaultdict(lambda:0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names_out()):
  word2id[feature] = idx

In [15]:
tfidf_vectorizer.vocabulary_

{'전력': 1762,
 '인맥': 1659,
 '미국': 734,
 '바탕': 770,
 '청산': 2010,
 '청사진': 2009,
 '논리': 425,
 '소망': 1128,
 '대결': 504,
 '현안': 2340,
 '좌절': 1828,
 '성과': 1089,
 '조기': 1800,
 '휘하': 2438,
 '글로벌': 302,
 '국민': 266,
 '슬기': 1200,
 '기억': 333,
 '새해': 1041,
 '갈등': 38,
 '사람': 973,
 '관행': 224,
 '무엇': 723,
 '기초': 346,
 '평화': 2208,
 '방향': 821,
 '도민': 557,
 '극복': 296,
 '다이': 465,
 '위기': 1532,
 '지혜': 1926,
 '역량': 1364,
 '번지': 838,
 '생명력': 1047,
 '대비': 519,
 '공감대': 172,
 '기반': 326,
 '관계': 209,
 '전환점': 1786,
 '당위': 501,
 '희망': 2444,
 '선진국': 1072,
 '협의': 2353,
 '다경': 456,
 '시련': 1214,
 '이것': 1608,
 '해다': 2296,
 '공유': 186,
 '포함': 2225,
 '인사': 1660,
 '그동안': 293,
 '치적': 2109,
 '역사': 1365,
 '퇴행': 2168,
 '낭비성': 397,
 '한미': 2274,
 '상화': 1035,
 '민족': 755,
 '아침': 1287,
 '여건': 1348,
 '열차': 1407,
 '대화': 543,
 '평창': 2205,
 '미래': 736,
 '우리': 1496,
 '사무소': 979,
 '햇살': 2313,
 '중요': 1872,
 '다음': 464,
 '건국': 89,
 '변수': 850,
 '조화': 1817,
 '현상': 2338,
 '불씨': 944,
 '열쇠': 1406,
 '흑인': 2441,
 '남북': 389,
 '금강산': 304,
 '출발': 2086,
 

In [16]:
tfidf_matrix.shape

(20, 2445)

In [20]:
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding= 'utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"---문서 {idx} 번 ---")
    print(line[:200])
    print(" ")

---문서 0 번 ---
새로운 희망 공유하고 새 출발하자 기축년(己丑年) 새해다.새로운 희망을 공유하고 새 출발을 다짐할 때다.짙푸른 동해바다를 뚫고 이글거리는 광채를 뿜으며 힘차게 솟아오르는 태양을 안는다. 어제와 다른 이 아침의 햇살을 받으며 왜 간절한 소망이 없겠는가.지난날의 갈등과 혼란 회한을 뒤로하고 새로 일어서야 한다.올해는 불신과 증오, 암투와 음모, 좌절과 절망 등
 
---문서 1 번 ---
<새해 이렇게 하겠습니다>스포츠 "최일선 현장에서 진솔한 소리 듣겠다" 최동용 도체육회 사무처장 지도자와 선수 등 체육의 최일선 현장에서 진솔한 소리를 더 많이 듣겠다. 강원체육이 지금까지 잘 할 수 있었던 것은 운동장 체육관 등 현장에서 지도자와 선수들이 묵묵히 운동한 결과다. 애로사항 등을 청취 체육정책에 반영하는 소통의 역할에 충실하겠다. 도청·도교육
 
---문서 2 번 ---
<새해 이렇게 하겠습니다>경제 '저탄소 녹색성장' 도모 전수산 도상공회의소협의회장 강원경제는 경제성장에 필요한 입지여건이나 기반조성이 타 지역보다 열악하고 최근 세계경제 침체의 영향으로 많은 어려움을 겪고 있다. 그러나 최근 수도권과의 교통망이 확충되면서 중앙 및 지방정부의 지역전략산업 육성 등 유리한 발전여건을 갖춰 나가고 있다. 정부의 5+2광역경제권 
 
---문서 3 번 ---
<새해 이렇게 하겠습니다>문화 이대근 강원도문화원연합회장 “지속적 향토사 연구로 우리 역사 되살리겠다” 어김없이 새로운 한 해가 밝았다. 지난해는 많은 시련을 안겨줬다. 선진국의 문턱에서 넘지 못하고 머뭇거리며 지나온 10년 세월이 원망스러운 가운데 이번에는 미국발 경제한파가 발목을 잡고 있다. 우리나라는 국민소득에서 보면 전 세계에서 24번째 국가에 머물
 
---문서 4 번 ---
<새해 이렇게 하겠습니다>속초 고성 양양 ■속초시 환동해 수출입항 메카 거듭 의정 옴부즈맨 제도 활성화 채용생 속초시장은 “올해는 민선4기를 사실상 마무리 짓는 해로 지금까지 성과를 바탕으로 성실한 마무리와 함께

In [27]:
for idx, sent in enumerate(all_sentences):
  print(f"---문서 {idx} 번 ---")
  results = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results.sort(key = lambda element : element[1], reverse = True)
  print(results, "\n")

---문서 0 번 ---
[('인맥', 0.08663733949674877), ('청산', 0.08663733949674877), ('청사진', 0.08663733949674877), ('소망', 0.08663733949674877), ('대결', 0.08663733949674877), ('휘하', 0.08663733949674877), ('기억', 0.08663733949674877), ('관행', 0.08663733949674877), ('평화', 0.08663733949674877), ('다이', 0.08663733949674877), ('번지', 0.08663733949674877), ('생명력', 0.08663733949674877), ('공감대', 0.08663733949674877), ('전환점', 0.08663733949674877), ('당위', 0.08663733949674877), ('다경', 0.08663733949674877), ('이것', 0.08663733949674877), ('해다', 0.08663733949674877), ('퇴행', 0.08663733949674877), ('낭비성', 0.08663733949674877), ('한미', 0.08663733949674877), ('상화', 0.08663733949674877), ('아침', 0.08663733949674877), ('열차', 0.08663733949674877), ('사무소', 0.08663733949674877), ('햇살', 0.08663733949674877), ('건국', 0.08663733949674877), ('불씨', 0.08663733949674877), ('열쇠', 0.08663733949674877), ('흑인', 0.08663733949674877), ('통행', 0.08663733949674877), ('취임', 0.08663733949674877), ('개성관광', 0.08663733949674877), ('치력', 0.08663733949